# compute.py

In [1]:
%%time
import pandas as pd
import re
from pymorphy2 import MorphAnalyzer
#from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

morph = MorphAnalyzer()
patterns_ru = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
patterns_kk = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
patterns_en = "[А-Яа-я0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
#stopwords_ru = stopwords.words("russian")
stopwords_ru=['другои', 'еи', 'какои', 'мои', 'неи', 'сеичас', 'такои', 'этои','и', 'в', 'во', 'не', 'что', 'он','на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне','было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между']
stopwords_en=['i','me', 'my', 'myself', 'we', 'our', 'ours','ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any','both', 'each', 'few', 'more', 'most', 'other','some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
stopwords_kk=['ах', 'ох', 'эх', 'ай', 'эй', 'ой', 'тағы', 'тағыда', 'әрине', 'жоқ', 'сондай', 'осындай', 'осылай', 'солай', 'мұндай', 'бұндай', 'мен', 'сен', 'ол', 'біз', 'біздер', 'олар', 'сіз', 'сіздер', 'маған', 'оған', 'саған', 'біздің', 'сіздің', 'оның', 'бізге', 'сізге','оларға', 'біздерге', 'сіздерге', 'оларға', 'менімен', 'сенімен', 'онымен', 'бізбен', 'сізбен', 'олармен', 'біздермен', 'сіздермен', 'менің', 'сенің', 'біздің', 'сіздің', 'оның', 'біздердің', 'сіздердің', 'олардың', 'маған', 'саған', 'оған', 'менен', 'сенен', 'одан', 'бізден', 'сізден', 'олардан', 'біздерден', 'сіздерден', 'олардан', 'айтпақшы', 'сонымен', 'сондықтан', 'бұл', 'осы', 'сол', 'анау', 'мынау', 'сонау', 'осынау', 'ана', 'мына', 'сона', 'әні', 'міне', 'өй', 'үйт', 'бүйт', 'біреу', 'кейбіреу', 'кейбір', 'қайсыбір', 'әрбір', 'бірнеше', 'бірдеме', 'бірнеше', 'әркім', 'әрне', 'әрқайсы', 'әрқалай', 'әлдекім', 'әлдене', 'әлдеқайдан', 'әлденеше', 'әлдеқалай', 'әлдеқашан', 'алдақашан', 'еш', 'ешкім', 'ешбір', 'ештеме', 'дәнеңе', 'ешқашан', 'ешқандай', 'ешқайсы', 'емес', 'бәрі', 'барлық', 'барша', 'бар', 'күллі', 'бүкіл', 'түгел', 'өз', 'өзім', 'өзің', 'өзінің', 'өзіме', 'өзіне', 'өзімнің', 'өзі', 'өзге', 'менде', 'сенде', 'онда', 'менен', 'сенен\tонан', 'одан', 'ау', 'па', 'ей', 'әй', 'е', 'уа', 'уау', 'уай', 'я', 'пай', 'ә', 'о', 'оһо', 'ой', 'ие', 'аһа', 'ау', 'беу', 'мәссаған', 'бәрекелді', 'әттегенай', 'жаракімалла', 'масқарай', 'астапыралла', 'япырмай', 'ойпырмай', 'кәне', 'кәнеки', 'ал', 'әйда', 'кәні', 'міне', 'әні', 'сорап', 'қош-қош', 'пфша', 'пішә', 'құрау-құрау', 'шәйт', 'шек', 'моһ', 'тәк', 'құрау', 'құр', 'кә', 'кәһ', 'күшім', 'күшім', 'мышы', 'пырс', 'әукім', 'алақай', 'паһ-паһ', 'бәрекелді', 'ура', 'әттең', 'әттеген-ай', 'қап', 'түге', 'пішту', 'шіркін', 'алатау', 'пай-пай', 'үшін', 'сайын', 'сияқты', 'туралы', 'арқылы', 'бойы', 'бойымен', 'шамалы', 'шақты', 'қаралы', 'ғұрлы', 'ғұрлым', 'шейін', 'дейін', 'қарай', 'таман', 'салым', 'тарта', 'жуық', 'таяу', 'гөрі', 'бері', 'кейін', 'соң', 'бұрын', 'бетер', 'қатар', 'бірге', 'қоса', 'арс', 'гүрс', 'дүрс', 'қорс', 'тарс', 'тырс', 'ырс', 'барқ', 'борт', 'күрт', 'кірт', 'морт', 'сарт', 'шырт', 'дүңк', 'күңк', 'қыңқ', 'мыңқ', 'маңқ', 'саңқ', 'шаңқ', 'шіңк', 'сыңқ', 'таңқ', 'тыңқ', 'ыңқ', 'болп','былп', 'жалп', 'желп', 'қолп', 'ірк', 'ырқ', 'сарт-сұрт', 'тарс-тұрс', 'арс-ұрс', 'жалт-жалт', 'жалт-жұлт', 'қалт-қалт', 'қалт-құлт', 'қаңқ-қаңқ', 'қаңқ-құңқ', 'шаңқ-шаңқ', 'шаңқ-шұңқ', 'арбаң-арбаң', 'бүгжең-бүгжең', 'арсалаң-арсалаң', 'ербелең-ербелең', 'батыр-бұтыр', 'далаң-далаң','тарбаң-тарбаң', 'қызараң-қызараң', 'қаңғыр-күңгір', 'қайқаң-құйқаң', 'митың-митың', 'салаң-сұлаң', 'ыржың-тыржың', 'бірақ', 'алайда', 'дегенмен', 'әйтпесе', 'әйткенмен', 'себебі', 'өйткені', 'сондықтан', 'үшін', 'сайын', 'сияқты', 'туралы', 'арқылы', 'бойы', 'бойымен', 'шамалы', 'шақты', 'қаралы', 'ғұрлы', 'ғұрлым', 'гөрі', 'бері', 'кейін', 'соң', 'бұрын', 'бетер', 'қатар', 'бірге', 'қоса', 'шейін', 'дейін', 'қарай', 'таман', 'салым', 'тарта', 'жуық', 'таяу', 'арнайы', 'осындай', 'ғана', 'қана', 'тек', 'әншейін']

def lemmatize_en(doc):
    doc = re.sub(patterns_en, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token.lower() not in stopwords_en:
            token = token.strip()
            #token = morph.normal_forms(token)[0]
            
            tokens.append(token.lower())
    if len(tokens) > 0:
        return tokens
    return None

def lemmatize_ru(doc):
    doc = re.sub(patterns_ru, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token.lower() not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            
            tokens.append(token.lower())
    if len(tokens) > 0:
        return tokens
    return None

def lemmatize_kk(doc):
    doc = re.sub(patterns_kk, ' ', doc)
    tokens = []
    for token in doc.split():
        if token and token.lower() not in stopwords_kk:
            token = token.strip()
            #token = morph.normal_forms(token)[0]
            
            tokens.append(token.lower())
    if len(tokens) > 0:
        return tokens
    return None

df_ru=pd.read_excel('catalog_ru.xlsx',sheet_name='1')
df_kk=pd.read_excel('catalog_kk.xlsx',sheet_name='1')
df_en=pd.read_excel('catalog_en.xlsx',sheet_name='1')

df2_ru=pd.read_excel('catalog2_ru.xlsx',sheet_name='1')
df2_kk=pd.read_excel('catalog2_kk.xlsx',sheet_name='1')
df2_en=pd.read_excel('catalog2_en.xlsx',sheet_name='1')

data_ru=df_ru['NAME'].apply(lemmatize_ru).str.join(' ').tolist()
data_kk=df_kk['NAME'].apply(lemmatize_kk).str.join(' ').tolist()
data_en=df_en['NAME'].apply(lemmatize_en).str.join(' ').tolist()

data2_ru=df2_ru['NAME'].apply(lemmatize_ru).str.join(' ').tolist()
data2_kk=df2_kk['NAME'].apply(lemmatize_kk).str.join(' ').tolist()
data2_en=df2_en['NAME'].apply(lemmatize_en).str.join(' ').tolist()

v = TfidfVectorizer(input='content',
                    encoding='utf-8', decode_error='replace', strip_accents='unicode',
                    lowercase=True, analyzer='word', stop_words=stopwords_kk,
                    #token_pattern=r'(?u)\b[а-яА-Я_][а-яА-Я0-9_]+\b',
                    #token_pattern=patterns_kk,
                    #ngram_range=(1, 2),
                    ngram_range=(1, 1),
                    max_features=20000,
                    norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=True,
                    max_df=0.1, min_df=5
                    #max_df=1, min_df=1
                   )

v2 = TfidfVectorizer(input='content',
                    encoding='utf-8', decode_error='replace', strip_accents='unicode',
                    lowercase=True, analyzer='word', stop_words=stopwords_kk,
                    #token_pattern=r'(?u)\b[а-яА-Я_][а-яА-Я0-9_]+\b',
                    #token_pattern=patterns_kk,
                    #ngram_range=(1, 2),
                    ngram_range=(1, 1),
                    max_features=20000,
                    norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=True,
                    max_df=0.1, min_df=5
                    #max_df=1, min_df=1
                   )


import pickle
data_kk=[x if x!=None else '' for x in data_kk]
tfidf_vectorizer_vectors=v.fit_transform(data_kk)
data_temp_kk=pd.DataFrame(tfidf_vectorizer_vectors.T.todense(),index=v.get_feature_names())#[1].sort_values(ascending=False).head(30)

pickle.dump(data_temp_kk,open('catalog_vec_tfidf_kk.pkl','wb'))
pickle.dump(v,open('model_tfidf_kk.pkl','wb'))
pickle.dump(df_kk,open('catalog_kk.pkl','wb'))

data2_kk=[x if x!=None else '' for x in data2_kk]
tfidf_vectorizer_vectors2=v2.fit_transform(data2_kk)
data2_temp_kk=pd.DataFrame(tfidf_vectorizer_vectors2.T.todense(),index=v2.get_feature_names())#[1].sort_values(ascending=False).head(30)

pickle.dump(data2_temp_kk,open('catalog2_vec_tfidf_kk.pkl','wb'))
pickle.dump(v2,open('model2_tfidf_kk.pkl','wb'))
pickle.dump(df2_kk,open('catalog2_kk.pkl','wb'))




pickle.dump(df_en,open('catalog_en.pkl','wb'))
pickle.dump(df_ru,open('catalog_ru.pkl','wb'))

pickle.dump(df2_en,open('catalog2_en.pkl','wb'))
pickle.dump(df2_ru,open('catalog2_ru.pkl','wb'))



from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine
import numpy as np
import os
import pandas as pd

model_transformer=SentenceTransformer(os.getcwd()+'\\'+'sentence-transformers_paraphrase-multilingual-mpnet-base-v2')

data_temp_en=model_transformer.encode(data_en)
data_temp_ru=model_transformer.encode(data_ru)

data2_temp_en=model_transformer.encode(data2_en)
data2_temp_ru=model_transformer.encode(data2_ru)


pickle.dump(data_temp_en,open('data_temp_en.pkl','wb'))
pickle.dump(data_temp_ru,open('data_temp_ru.pkl','wb'))

pickle.dump(data2_temp_en,open('data2_temp_en.pkl','wb'))
pickle.dump(data2_temp_ru,open('data2_temp_ru.pkl','wb'))

pickle.dump(model_transformer,open('model_transformer.pkl','wb'))


C:\Users\shaim\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['аи', 'аитпақшы', 'алаида', 'алақаи', 'арбаң', 'арнаиы', 'арсалаң', 'батыр', 'боиы', 'боиымен', 'бүгжең', 'бүит', 'бұндаи', 'бұтыр', 'далаң', 'деиін', 'еи', 'ербелең', 'ешқаисы', 'ешқандаи', 'жалт', 'жұлт', 'кеибір', 'кеибіреу', 'кеиін', 'күңгір', 'масқараи', 'митың', 'мұндаи', 'ои', 'оипырмаи', 'онан', 'осылаи', 'осындаи', 'паи', 'паһ', 'саиын', 'салаң', 'солаи', 'сондаи', 'сұлаң', 'сұрт', 'тарбаң', 'тыржың', 'тұрс', 'уаи', 'шеиін', 'шұңқ', 'шәит', 'ыржың', 'эи', 'япырмаи', 'қаисыбір', 'қаиқаң', 'қалт', 'қараи', 'қаңғыр', 'қаңқ', 'қош', 'қызараң', 'құиқаң', 'құлт', 'құңқ', 'үит', 'ұрс', 'әи', 'әида', 'әиткенмен', 'әитпесе', 'әлдеқаидан', 'әлдеқалаи', 'әншеиін', 'әрқаисы', 'әрқалаи', 'әттеген', 'әттегенаи', 'өи', 'өиткені'] not in stop_words.
  warnings.warn('Your stop_words may be incons

Wall time: 3min 23s


In [7]:
#[i for i in data_temp_kk.index.tolist() if 'шыр' in i]

In [ ]:
1

# web_server.py

In [4]:
from flask import Flask, request, render_template
import pickle
import time
from pymorphy2 import MorphAnalyzer
#import fasttext
#model = fasttext.load_model('lid.176.ftz')
from scipy.spatial.distance import cosine
import numpy as np
import os
import pandas as pd
import re


model_transformer=pickle.load(open('model_transformer.pkl','rb'))

data_temp_ru=pickle.load(open('data_temp_ru.pkl','rb'))
df_ru=pickle.load(open('catalog_ru.pkl','rb'))

data2_temp_ru=pickle.load(open('data2_temp_ru.pkl','rb'))
df2_ru=pickle.load(open('catalog2_ru.pkl','rb'))


data_temp_en=pickle.load(open('data_temp_en.pkl','rb'))
df_en=pickle.load(open('catalog_en.pkl','rb'))

data2_temp_en=pickle.load(open('data2_temp_en.pkl','rb'))
df2_en=pickle.load(open('catalog2_en.pkl','rb'))

data_temp_kk=pickle.load(open('catalog_vec_tfidf_kk.pkl','rb'))
v_kk=pickle.load(open('model_tfidf_kk.pkl','rb'))
df_kk=pickle.load(open('catalog_kk.pkl','rb'))

data2_temp_kk=pickle.load(open('catalog2_vec_tfidf_kk.pkl','rb'))
v2_kk=pickle.load(open('model2_tfidf_kk.pkl','rb'))
df2_kk=pickle.load(open('catalog2_kk.pkl','rb'))

patterns_ru = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
patterns_kk = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
patterns_en = "[А-Яа-я0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
#stopwords_ru = stopwords.words("russian")
stopwords_ru=['другои', 'еи', 'какои', 'мои', 'неи', 'сеичас', 'такои', 'этои','и', 'в', 'во', 'не', 'что', 'он','на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне','было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между']
stopwords_en=['i','me', 'my', 'myself', 'we', 'our', 'ours','ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any','both', 'each', 'few', 'more', 'most', 'other','some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
stopwords_kk=['ах', 'ох', 'эх', 'ай', 'эй', 'ой', 'тағы', 'тағыда', 'әрине', 'жоқ', 'сондай', 'осындай', 'осылай', 'солай', 'мұндай', 'бұндай', 'мен', 'сен', 'ол', 'біз', 'біздер', 'олар', 'сіз', 'сіздер', 'маған', 'оған', 'саған', 'біздің', 'сіздің', 'оның', 'бізге', 'сізге','оларға', 'біздерге', 'сіздерге', 'оларға', 'менімен', 'сенімен', 'онымен', 'бізбен', 'сізбен', 'олармен', 'біздермен', 'сіздермен', 'менің', 'сенің', 'біздің', 'сіздің', 'оның', 'біздердің', 'сіздердің', 'олардың', 'маған', 'саған', 'оған', 'менен', 'сенен', 'одан', 'бізден', 'сізден', 'олардан', 'біздерден', 'сіздерден', 'олардан', 'айтпақшы', 'сонымен', 'сондықтан', 'бұл', 'осы', 'сол', 'анау', 'мынау', 'сонау', 'осынау', 'ана', 'мына', 'сона', 'әні', 'міне', 'өй', 'үйт', 'бүйт', 'біреу', 'кейбіреу', 'кейбір', 'қайсыбір', 'әрбір', 'бірнеше', 'бірдеме', 'бірнеше', 'әркім', 'әрне', 'әрқайсы', 'әрқалай', 'әлдекім', 'әлдене', 'әлдеқайдан', 'әлденеше', 'әлдеқалай', 'әлдеқашан', 'алдақашан', 'еш', 'ешкім', 'ешбір', 'ештеме', 'дәнеңе', 'ешқашан', 'ешқандай', 'ешқайсы', 'емес', 'бәрі', 'барлық', 'барша', 'бар', 'күллі', 'бүкіл', 'түгел', 'өз', 'өзім', 'өзің', 'өзінің', 'өзіме', 'өзіне', 'өзімнің', 'өзі', 'өзге', 'менде', 'сенде', 'онда', 'менен', 'сенен\tонан', 'одан', 'ау', 'па', 'ей', 'әй', 'е', 'уа', 'уау', 'уай', 'я', 'пай', 'ә', 'о', 'оһо', 'ой', 'ие', 'аһа', 'ау', 'беу', 'мәссаған', 'бәрекелді', 'әттегенай', 'жаракімалла', 'масқарай', 'астапыралла', 'япырмай', 'ойпырмай', 'кәне', 'кәнеки', 'ал', 'әйда', 'кәні', 'міне', 'әні', 'сорап', 'қош-қош', 'пфша', 'пішә', 'құрау-құрау', 'шәйт', 'шек', 'моһ', 'тәк', 'құрау', 'құр', 'кә', 'кәһ', 'күшім', 'күшім', 'мышы', 'пырс', 'әукім', 'алақай', 'паһ-паһ', 'бәрекелді', 'ура', 'әттең', 'әттеген-ай', 'қап', 'түге', 'пішту', 'шіркін', 'алатау', 'пай-пай', 'үшін', 'сайын', 'сияқты', 'туралы', 'арқылы', 'бойы', 'бойымен', 'шамалы', 'шақты', 'қаралы', 'ғұрлы', 'ғұрлым', 'шейін', 'дейін', 'қарай', 'таман', 'салым', 'тарта', 'жуық', 'таяу', 'гөрі', 'бері', 'кейін', 'соң', 'бұрын', 'бетер', 'қатар', 'бірге', 'қоса', 'арс', 'гүрс', 'дүрс', 'қорс', 'тарс', 'тырс', 'ырс', 'барқ', 'борт', 'күрт', 'кірт', 'морт', 'сарт', 'шырт', 'дүңк', 'күңк', 'қыңқ', 'мыңқ', 'маңқ', 'саңқ', 'шаңқ', 'шіңк', 'сыңқ', 'таңқ', 'тыңқ', 'ыңқ', 'болп','былп', 'жалп', 'желп', 'қолп', 'ірк', 'ырқ', 'сарт-сұрт', 'тарс-тұрс', 'арс-ұрс', 'жалт-жалт', 'жалт-жұлт', 'қалт-қалт', 'қалт-құлт', 'қаңқ-қаңқ', 'қаңқ-құңқ', 'шаңқ-шаңқ', 'шаңқ-шұңқ', 'арбаң-арбаң', 'бүгжең-бүгжең', 'арсалаң-арсалаң', 'ербелең-ербелең', 'батыр-бұтыр', 'далаң-далаң','тарбаң-тарбаң', 'қызараң-қызараң', 'қаңғыр-күңгір', 'қайқаң-құйқаң', 'митың-митың', 'салаң-сұлаң', 'ыржың-тыржың', 'бірақ', 'алайда', 'дегенмен', 'әйтпесе', 'әйткенмен', 'себебі', 'өйткені', 'сондықтан', 'үшін', 'сайын', 'сияқты', 'туралы', 'арқылы', 'бойы', 'бойымен', 'шамалы', 'шақты', 'қаралы', 'ғұрлы', 'ғұрлым', 'гөрі', 'бері', 'кейін', 'соң', 'бұрын', 'бетер', 'қатар', 'бірге', 'қоса', 'шейін', 'дейін', 'қарай', 'таман', 'салым', 'тарта', 'жуық', 'таяу', 'арнайы', 'осындай', 'ғана', 'қана', 'тек', 'әншейін']

morph = MorphAnalyzer()    

app = Flask(__name__)

def result_v1(search_text,data_temp=None,v=None,df=None):

    
    #from sklearn.metrics.pairwise import cosine_similarity
    from scipy.spatial.distance import cosine
    import numpy as np
    import pandas as pd
    import re
    
    


    def lemmatize_kk(doc):
        doc = re.sub(patterns_kk, ' ', doc)
        tokens = []
        for token in doc.split():
            if token and token.lower() not in stopwords_kk:
                token = token.strip()
                #token = morph.normal_forms(token)[0]

                tokens.append(token.lower())
        if len(tokens) > 0:
            return tokens
        return None


    a=[search_text]
    try:
        a=[' '.join(pd.Series(a).apply(lemmatize_kk)[0])]
        
    except Exception:
        a=['null']
        pass
    
    a_vec=np.array(v.transform(a).T.todense()).flatten()
    t=[1-cosine(a_vec, np.array((data_temp[i]))) for i in range(data_temp.shape[1])]

#     from joblib import Parallel, delayed
#     def process(i):
#         return cosine(a_vec,i)
#     t = Parallel(n_jobs=2)(delayed(process)(i) for i in range(data_temp.shape[1]))


    t = np.array(t)
    
    
    inds = (-t).argsort()
    
    top=10
    source2=df['SOURCE'].loc[inds[:top]].tolist()
    code2=df['CODE'].loc[inds[:top]].tolist()
    name2=df['NAME'].loc[inds[:top]].tolist()
    descr2=df['DESCRIPTION'].loc[inds[:top]].tolist()
    score2=t[inds[:top]].tolist()
    
    cutoff=np.where(np.array(score2)>0.5,True,False)
    
    source2=[val for is_good, val in zip(cutoff, source2) if is_good]
    code2=[val for is_good, val in zip(cutoff, code2) if is_good]
    name2=[val for is_good, val in zip(cutoff, name2) if is_good]
    score2=[val for is_good, val in zip(cutoff, score2) if is_good]
    descr2=[val for is_good, val in zip(cutoff, descr2) if is_good]


    return source2,code2,name2,descr2,score2

def models_SentenceTransformer(search_text,model,patterns,stopwords,df,data_temp):
    
    
    def lemmatize(doc,patterns,stopwords):
        doc = re.sub(patterns, ' ', doc)
        tokens = []
        for token in doc.split():
            if token and token not in stopwords:
                token = token.strip()
                token = morph.normal_forms(token)[0]

                tokens.append(token.lower())
        if len(tokens) > 0:
            return tokens
        return None

    
    if search_text=='':
        search_text='пример'

    
    search_text=[search_text]
    search_text=[' '.join(pd.Series(search_text).apply(lambda x: lemmatize(x,patterns,stopwords))[0])]

    a_vec=model.encode(search_text)

    import time
    
    for i in range(1):
        t=[1-cosine(a_vec, np.array((data_temp[i]))) for i in range(df.shape[0])] 
    


    t = np.array(t)

    inds = (-t).argsort()

    top=10
    source2=df['SOURCE'].loc[inds[:top]].tolist()
    code2=df['CODE'].loc[inds[:top]].tolist()
    name2=df['NAME'].loc[inds[:top]].tolist()
    descr2=df['DESCRIPTION'].loc[inds[:top]].tolist()
    score2=t[inds[:top]].tolist()

    cutoff=np.where(np.array(score2)>0.5,True,False)

    source2=[val for is_good, val in zip(cutoff, source2) if is_good]
    code2=[val for is_good, val in zip(cutoff, code2) if is_good]
    name2=[val for is_good, val in zip(cutoff, name2) if is_good]
    score2=[val for is_good, val in zip(cutoff, score2) if is_good]
    descr2=[val for is_good, val in zip(cutoff, descr2) if is_good]
    

    return source2,code2,name2,descr2,score2


from flask import Flask, request, render_template
import time
app = Flask(__name__)
#import fasttext
#model = fasttext.load_model('lid.176.ftz')

@app.route('/', methods=['POST','GET'])    
def my_form_post():
    
    start = time.time()
    time.sleep(0.3)

    a='пример'
    
    if request.method=="POST":
        a=request.form.get('name2')
        

#     predict_language=model.predict(a, k=20)
#     predict_language_lan,_=predict_language
#     for i in predict_language_lan:
#         if i in ['__label__en','__label__ru','__label__kk']:
#             result=i[-2:]
#             break
    result='ru'
    
    
    if result=='ru':
        source,code,name,descr,score=models_SentenceTransformer(a,model_transformer,patterns_ru,stopwords_ru,df_ru,data_temp_ru)
        source2,code2,name2,descr2,score2=models_SentenceTransformer(a,model_transformer,patterns_ru,stopwords_ru,df2_ru,data2_temp_ru)      
    elif result=='kk':
        source,code,name,descr,score=result_v1(a,data_temp_kk,v_kk,df_kk)
        source2,code2,name2,descr2,score2result_v1(a,data2_temp_kk,v2_kk,df2_kk)
    elif result=='en':
        source,code,name,descr,score=models_SentenceTransformer('jackets',model_transformer,patterns_en,stopwords_en,df_en,data_temp_en) 
        source2,code2,name2,descr2,score2=models_SentenceTransformer('jackets',model_transformer,patterns_en,stopwords_en,df2_en,data2_temp_en)
        
        
        
    select = request.form.get('comp_select')
    
    source0=source+source2
    code0=code+code2
    name0=name+name2
    score0=score+score2
    descr0=descr+descr2
    
    #code0=[a for (a, truth) in zip([i['code'] for i in code0], [i['name']=='пример2' for i in code0]) if truth]
    
    
    
    keys=['source','name','code','description','score']
    values=[source0,name0,code0,descr0,score0]

    newList = []
    for i in range(len(values[0])):

        tmp = {}
        for j, key in enumerate(keys):
            tmp[key] = values[j][i]

        newList.append(tmp)
    

    if select=='s1':
        newList=[val for is_good, val in zip([i['source']=='TNVED' for i in newList], newList) if is_good]
    elif select=='s2':
        newList=[val for is_good, val in zip([i['source']=='ENSTRU' for i in newList], newList) if is_good]       
        
    end = time.time()

    return render_template('1.html',data=newList,time_calc=round(end-start,3))        




if __name__=='__main__':
    app.run(host='0.0.0.0', port=86, debug=False)

In [4]:
%%time
result_v1('шырыны',data_temp_kk,v_kk,df_kk)

Wall time: 472 ms


(['TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED'],
 [200950,
  200949,
  200979,
  200919,
  200911,
  200929,
  200981,
  200969,
  200921,
  200961,
  200941,
  200971],
 ['Томат шырыны',
  'Басқа ананас шырыны',
  'Басқа алма шырыны',
  'Апельсин шырыны, т.б',
  'мұздатылған апельсин шырыны',
  'Грейпфрут шырыны, помело шырыны және т.б',
  'Мүкжидек шырыны (vaccinium macrocarpon, vaccinium oxycoccos, vaccinium vitis-идея), лингонжидек шырыны (Vaccinium vitis-idaea)',
  'Басқа жүзім шырыны (соның ішінде жүзім шырыны)',
  'Грейпфрут шырыны; помело шырыны, Brix 20-дан аспайды',
  'Брикс 30-дан аспайтын жүзім шырыны (соның ішінде жүзім шырыны).',
  'Brix мәні 20-дан аспайтын ананас шырыны',
  'Brix мәні 20-дан аспайтын алма шырыны'],
 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
 [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.8737967900348175,
  0.8087949702572941,
  0.782350756505348

In [5]:
%%time
models_SentenceTransformer('jackets',model_transformer,patterns_en,stopwords_en,df_en,data_temp_en)

Wall time: 364 ms


(['TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED'],
 [620190,
  610190,
  610120,
  620130,
  620290,
  610290,
  620120,
  610210,
  621600,
  620240,
  620140,
  630720,
  620230,
  610220,
  610230,
  610130,
  420329,
  610333,
  611699,
  621120,
  620220,
  611610,
  620339,
  610332,
  481850,
  611220,
  420321,
  620333,
  620439,
  611011,
  611692,
  611090,
  611019,
  640212,
  620433,
  630190,
  640312,
  610339,
  6

In [6]:
%%time
models_SentenceTransformer('молоко',model_transformer,patterns_ru,stopwords_ru,df_ru,data_temp_ru)

Wall time: 388 ms


(['TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',
  'TNVED',

In [7]:
%%time
result_v1('шырыны',data2_temp_kk,v2_kk,df2_kk)

Wall time: 774 ms


([], [], [], [], [])

In [8]:
%%time
models_SentenceTransformer('jackets',model_transformer,patterns_en,stopwords_en,df2_en,data2_temp_en)

Wall time: 528 ms


(['ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU'],
 ['222929.500.000006',
  '259912.400.000000',
  '162914.900.000011',
  '171220.900.000000',
  '262040.000.000119',
  '139229.990.000004',
  '141230.100.000019',
  '141230.100.000018',
  '141230.100.000017',
  '141230.100.000015',
  '141230.100.000013',
  '141230.100.000009',
  '141932.350.000005',
  '141230.100.000008',
  '141230.100.000003',
  '141230.100.000002',
  '141230.100.000001',
  '141230.100.000010',
  '141932.350.000006',
  '141923.700.000009',
  '221960.590.000000',
  '141932.350.000007',
  '151212.900.000083',
  '222925.900.000016',
  '222925.900.000015',
  '222925.700.000036',
  '222925.700.000034',
  '151212.900.0000

In [9]:
%%time
models_SentenceTransformer('молоток',model_transformer,patterns_ru,stopwords_ru,df2_ru,data2_temp_ru)

Wall time: 536 ms


(['ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'ENSTRU',
  'E

In [5]:
from flask import Flask, request, render_template
import time
app = Flask(__name__)
#import fasttext
#model = fasttext.load_model('lid.176.ftz')

@app.route('/', methods=['POST','GET'])    
def my_form_post():
    
    start = time.time()
    time.sleep(0.3)

    a='пример'
    
    if request.method=="POST":
        a=request.form.get('name2')
        

#     predict_language=model.predict(a, k=20)
#     predict_language_lan,_=predict_language
#     for i in predict_language_lan:
#         if i in ['__label__en','__label__ru','__label__kk']:
#             result=i[-2:]
#             break
    result='ru'
    
    
    if result=='ru':
        source,code,name,descr,score=models_SentenceTransformer(a,model_transformer,patterns_ru,stopwords_ru,df_ru,data_temp_ru)
        source2,code2,name2,descr2,score2=models_SentenceTransformer(a,model_transformer,patterns_ru,stopwords_ru,df2_ru,data2_temp_ru)      
    elif result=='kk':
        source,code,name,descr,score=result_v1(a,data_temp_kk,v_kk,df_kk)
        source2,code2,name2,descr2,score2result_v1(a,data2_temp_kk,v2_kk,df2_kk)
    elif result=='en':
        source,code,name,descr,score=models_SentenceTransformer('jackets',model_transformer,patterns_en,stopwords_en,df_en,data_temp_en) 
        source2,code2,name2,descr2,score2=models_SentenceTransformer('jackets',model_transformer,patterns_en,stopwords_en,df2_en,data2_temp_en)
        
        
        
    select = request.form.get('comp_select')
    
    source0=source+source2
    code0=code+code2
    name0=name+name2
    score0=score+score2
    descr0=descr+descr2
    
    #code0=[a for (a, truth) in zip([i['code'] for i in code0], [i['name']=='пример2' for i in code0]) if truth]
    
    
    
    keys=['source','name','code','description','score']
    values=[source0,name0,code0,descr0,score0]

    newList = []
    for i in range(len(values[0])):

        tmp = {}
        for j, key in enumerate(keys):
            tmp[key] = values[j][i]

        newList.append(tmp)
    

    if select=='s1':
        newList=[val for is_good, val in zip([i['source']=='TNVED' for i in newList], newList) if is_good]
    elif select=='s2':
        newList=[val for is_good, val in zip([i['source']=='ENSTRU' for i in newList], newList) if is_good]       
        
    end = time.time()

    return render_template('1.html',data=newList,time_calc=round(end-start,3))        




if __name__=='__main__':
    app.run(host='0.0.0.0', port=86, debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:86/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Sep/2022 21:24:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 21:24:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 21:24:46] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 21:24:50] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 21:24:55] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 21:25:01] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2022 21:25:09] "POST / HTTP/1.1" 200 -


In [7]:
from sentence_transformers import SentenceTransformer

In [10]:
SentenceTransformer

sentence_transformers.SentenceTransformer.SentenceTransformer

In [ ]:
sentence-transformers==2.2.2
torch==1.11.0

In [ ]:
pip3 install torch==1.8.2 torchvision==0.9.2 torchaudio==0.8.2 --extra-index-url https://download.pytorch.org/whl/lts/1.8/cpu

In [20]:
def d1(select):

    name=[select,'пример2','пример2']
    code=[1,123,456]
    score=[0.3,0.5,0.9]
    
    keys=['name','code','score']
    values=[name,code,score]

    newList = []
    for i in range(len(values[0])):

        tmp = {}
        for j, key in enumerate(keys):
            tmp[key] = values[j][i]

        newList.append(tmp)
    return newList

In [21]:
a1=d1('ss')
a2=d1('ss2')

In [34]:
a1+a2

[{'name': 'ss', 'code': 1, 'score': 0.3},
 {'name': 'пример2', 'code': 123, 'score': 0.5},
 {'name': 'пример2', 'code': 456, 'score': 0.9},
 {'name': 'ss2', 'code': 1, 'score': 0.3},
 {'name': 'пример2', 'code': 123, 'score': 0.5},
 {'name': 'пример2', 'code': 456, 'score': 0.9}]

In [35]:
[i['name']=='пример2' for i in a1+a2]

[False, True, True, False, True, True]

In [36]:
b1=a1+a2

In [37]:
[b1[i] for i in [i['name']=='пример2' for i in b1]]

[{'name': 'ss', 'code': 1, 'score': 0.3},
 {'name': 'пример2', 'code': 123, 'score': 0.5},
 {'name': 'пример2', 'code': 123, 'score': 0.5},
 {'name': 'ss', 'code': 1, 'score': 0.3},
 {'name': 'пример2', 'code': 123, 'score': 0.5},
 {'name': 'пример2', 'code': 123, 'score': 0.5}]

In [38]:
[i['name']=='пример2' for i in b1]

[False, True, True, False, True, True]

In [39]:
b1

[{'name': 'ss', 'code': 1, 'score': 0.3},
 {'name': 'пример2', 'code': 123, 'score': 0.5},
 {'name': 'пример2', 'code': 456, 'score': 0.9},
 {'name': 'ss2', 'code': 1, 'score': 0.3},
 {'name': 'пример2', 'code': 123, 'score': 0.5},
 {'name': 'пример2', 'code': 456, 'score': 0.9}]

[{'name': 'пример2', 'code': 123, 'score': 0.5},
 {'name': 'пример2', 'code': 456, 'score': 0.9},
 {'name': 'пример2', 'code': 123, 'score': 0.5},
 {'name': 'пример2', 'code': 456, 'score': 0.9}]

In [ ]:
[a for (a, truth) in zip([i['name'] for i in a1+a2], [i['name']=='пример2' for i in a1+a2]) if truth]

In [15]:
[a for (a, truth) in zip([i['name'] for i in a1+a2], [i['name']=='пример2' for i in a1+a2]) if truth]

['пример2', 'пример2', 'пример2', 'пример2']

In [16]:
[a for (a, truth) in zip([i['code'] for i in a1+a2], [i['name']=='пример2' for i in a1+a2]) if truth]

[123, 456, 123, 456]

In [18]:
b1

NameError: name 'b1' is not defined